In [1]:
import pandas as pd

In [2]:
topic = pd.read_csv('C:/Users/user/Desktop/topic_sample.csv')
peak = pd.read_csv('C:/Users/user/Desktop/peak_sample.csv')

In [53]:
topic.head()

,date,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
0,2010-01-01,11.0,22.0,4.0,5.0,4.0,3.0,0.0,21.0,0.0
1,2010-01-02,4.0,53.0,5.0,2.0,4.0,5.0,2.0,7.0,54.0
2,2010-01-03,22.0,31.0,9.0,2.0,8.0,6.0,5.0,7.0,54.0
3,2010-01-04,7.0,85.0,1.0,0.0,4.0,4.0,5.0,8.0,7.0
4,2010-01-05,9.0,2.0,2.0,1.0,2.0,3.0,4.0,8.0,7.0


In [54]:
peak

,date,volume
0,2010-01-10,9984561
1,2010-01-18,6948645
2,2010-01-22,5484821
3,2010-01-23,5545484
4,2010-01-24,4512161
5,2010-01-29,2545455


In [55]:
peak_topic = pd.merge(topic, peak, on='date', how='inner')
peak_topic = peak_topic.set_index('date')
del peak_topic['volume']
peak_topic

,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9
date,,,,,,,,,
2010-01-10,9.0,5.0,2.0,2.0,54.0,5.0,4.0,8.0,7.0
2010-01-18,22.0,31.0,9.0,2.0,8.0,7.0,5.0,7.0,54.0
2010-01-22,4.0,5.0,5.0,2.0,8.0,5.0,2.0,7.0,54.0
2010-01-23,22.0,31.0,9.0,2.0,45.0,6.0,5.0,4.0,54.0
2010-01-24,7.0,85.0,1.0,0.0,4.0,4.0,5.0,8.0,7.0
2010-01-29,7.0,85.0,1.0,0.0,4.0,4.0,5.0,8.0,7.0


In [56]:
df = pd.DataFrame(peak_topic.sum(axis=0), columns=['count'])
df

,count
topic1,71.0
topic2,242.0
topic3,27.0
topic4,8.0
topic5,123.0
topic6,31.0
topic7,26.0
topic8,42.0
topic9,183.0


In [57]:
df['fve']=''
for i in range(len(df)):
    df['fve'][i]=df['count'][i]/sum(df['count'])
df.sort_values(by='fve' ,ascending=False, inplace=True)

<ipython-input-57-ac8023a10a85>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fve'][i]=df['count'][i]/sum(df['count'])


In [58]:
df

,count,fve
topic2,242.0,0.321381
topic9,183.0,0.243028
topic5,123.0,0.163347
topic1,71.0,0.0942895
topic8,42.0,0.0557769
topic6,31.0,0.0411687
topic3,27.0,0.0358566
topic7,26.0,0.0345286
topic4,8.0,0.0106242


In [59]:
df['cumsum'] = df.fve.cumsum()
df

,count,fve,cumsum
topic2,242.0,0.321381,0.321381
topic9,183.0,0.243028,0.564409
topic5,123.0,0.163347,0.727756
topic1,71.0,0.0942895,0.822045
topic8,42.0,0.0557769,0.877822
topic6,31.0,0.0411687,0.918991
topic3,27.0,0.0358566,0.954847
topic7,26.0,0.0345286,0.989376
topic4,8.0,0.0106242,1


In [60]:
final = df['cumsum'] < 0.9
new_df = df[final]
new_df

,count,fve,cumsum
topic2,242.0,0.321381,0.321381
topic9,183.0,0.243028,0.564409
topic5,123.0,0.163347,0.727756
topic1,71.0,0.0942895,0.822045
topic8,42.0,0.0557769,0.877822


In [45]:
def get_FVE(topic, peak, n):
    
    peak_topic = pd.merge(topic, peak, on='date', how='inner')
    peak_topic = peak_topic.set_index('date')
    del peak_topic['volume']
    
    # 토픽별로 합계 테이블 작성
    df = pd.DataFrame(peak_topic.sum(axis=0), columns=['count'])
    
    # fve 칼럼 생성 후 내림차순 정렬
    df['fve']=''
    for i in range(len(df)):
        df['fve'][i]=df['count'][i]/sum(df['count'])
    df.sort_values(by='fve' ,ascending=False, inplace=True)
    
    # 누적합 칼럼 생성
    df['cumsum'] = df.fve.cumsum()
    
    # 누적합 n미만인 칼럼 추출
    final = df['cumsum'] < n
    new_df = df[final]
    
    return new_df

In [61]:
get_FVE(topic, peak, 0.9)

<ipython-input-45-4a51f7ab7ebd>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['fve'][i]=df['count'][i]/sum(df['count'])


,count,fve,cumsum
topic2,242.0,0.321381,0.321381
topic9,183.0,0.243028,0.564409
topic5,123.0,0.163347,0.727756
topic1,71.0,0.0942895,0.822045
topic8,42.0,0.0557769,0.877822
